In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import pandas
import rasterio
from osgeo import gdal
import geopandas as gpd
from sklearn.linear_model import LinearRegression
import os
import georasters as gr
import datetime as dt
from statsmodels.tsa.seasonal import STL

/home/luckyw0w/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
directory = 'download_ssm/cog/'
files = os.listdir(directory)
files.sort()

In [4]:
i = files[100][17:25]
i = i[:4]+'-'+i[4:6]+'-'+i[6:]
i
df = pd.DataFrame()
for i in files:
    myRaster = directory+i
    elevation = gr.from_file(myRaster)
    elevation = elevation.to_pandas()
    i = i[17:25]
    i = i[:4]+'-'+i[4:6]+'-'+i[6:]
    df[i] = elevation['value']
#df['geometry'] = elevation['geometry']

df = df.T
df['Date'] = df.index
df['Date'] = pd.to_datetime(df['Date'])
DateOrdinal = [dt.datetime.toordinal(i) for i in df['Date']]
df = df.reset_index()
del df['index']

In [18]:
def Trend_score(i, df, DateOrdinal):
    sdf = pd.Series(list(df[i]), index=pd.to_datetime(df['Date']), name="Fires")
    stl = STL(sdf)
    res = stl.fit()

    #Set the data in the right format for Linear Regression
    x = np.array(DateOrdinal)
    X = x.reshape(-1, 1)
    y = np.array(res.trend)
    y = y.reshape(-1, 1)

    #Perform Linear Regression and obtain the slope
    reg = LinearRegression().fit(X, y)
    y_pred_trend = reg.predict(X)
    slope, intercept = np.polyfit(x, y_pred_trend,1)
    
    return slope[0]

In [22]:
slopes = [Trend_score(i, df, DateOrdinal) for i in df.columns[:-1]]

In [26]:
res = np.array(slopes)
data_norm = np.where(res >= 0, res/np.max(res), -res/np.min(res))

In [24]:
elevation = gr.from_file(myRaster)
elevation = elevation.to_geopandas()

In [37]:
deviance = gpd.GeoDataFrame()
#deviance['index'] = elevation.index
deviance['deviance'] = data_norm
deviance['geometry'] = elevation['geometry']

In [38]:
deviance.to_file('SSM_deviance_10km.json')

In [40]:
data_norm.max()

1.0